##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 구조적인 잘라내기를 사용한 희소한 가중치

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/g3doc/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/g3doc/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소그 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/model-optimization/tensorflow_model_optimization/g3doc/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

특정 패턴에서 희소하도록 모델의 구조적인 잘라내기 가중치를 사용하면 적절한 HW 지원으로 모델 추론 시간을 가속할 수 있습니다.

이 튜토리얼은 다음과 같은 작업을 하는 방법을 설명합니다.

- 특정한 구조적인 희소성으로 MNIST 데이터세트에서 모델 정의 및 훈련하기
- 잘라내기 된 모델을 tflite 형식으로 변환하기
- 잘라내기 된 가중치 구조 시각화

모델 최적화를 위한 잘라내기 기술에 대한 일반적인 개요는 [잘라내기 개요](https://www.tensorflow.org/model_optimization/guide/pruning)를 참조하세요. 일반적인 가중치 잘라내기에 대한 튜토리얼의 경우, [Keras에서 잘라내기](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras)를 참조하세요.

## 가중치의 구조적 잘라내기

구조적 잘라내기는 트레이닝 프로세스 시작 시 모델 가중치를 체계적으로 0으로 만듭니다. 이 잘라내기 기술을 가중치의 정규 블록에 적용하여 HW를 지원하자마자 추론의 속도를 높입니다. (예: 4개의 블록으로 모델 내 가중치를 그룹화하고 각 블록의 가중치 중 2개를 0으로 만듭니다. *2 X 4* 감소로 알려져 있습니다.) 이 기술은 TensorFlow Lite로 변환되는 모델에 대한 가중치 텐서의 마지막 차원에만 적용됩니다. 예를 들어, TensorFlow Lite의 `Conv2D` 레이어 가중치는 `[channel_out, height, width, channel_in]`의 구조를 가지고 있고 `Dense` 레이어 가중치는 `[channel_out, channel_in]`의 구조를 가지고 있습니다. 희소성 패턴은 마지막 차원의 가중치에 적용됩니다.(`channel_in`)

무작위 희소성과 비교하여, 구조화된 희소성은 일반적으로 제한적인 구조로 인해 정확성이 낮지만, 지원되는 하드웨어에서 추론 시간을 상당히 줄일 수 있습니다.

잘라내기는 보다 나은 압축률을 위해 다른 모델 압축 기술과 함께 모델에 적용될 수 있습니다. 더 자세한 내용은 [공동 작업 최적화 기술](https://blog.tensorflow.org/2021/10/Collaborative-Optimizations.html)의 양자화 및 클러스터링 예시를 참조하세요.

## 설치

개발 환경과 데이터를 준비합니다.

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization
! pip install -q matplotlib

In [ ]:
import tensorflow as tf
from tensorflow import keras

import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

## [MNIST](https://www.tensorflow.org/datasets/catalog/mnist) 데이터세트에서 이미지 데이터를 다운로드하고 정규화하기

In [ ]:
# Load MNIST dataset.
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

## 구조적 잘라내기 매개변수 정의하기

구조적 잘라내기 유형을 잘라내고 지정하기 위해 매개변수를 정의합니다. 잘라내기에 대한 매개변수를 `(2, 4)`로 설정합니다. 이러한 설정은 4개 블록 요소에서 최소 두 개의 규모가 0으로 설정됨을 의미합니다.

매개변수를 설정할 필요는 없습니다. 기본적으로 잘라내기 마스크는 첫 단계에서 정의되며 훈련 중에는 업데이트 되지 않습니다.

In [ ]:
pruning_params_2_by_4 = {
    'sparsity_m_by_n': (2, 4),
}

대상 희소성 50%로 무작위 잘라내기를 위한 매개변수를 정의합니다.

In [ ]:
pruning_params_sparsity_0_5 = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.5,
                                                              begin_step=0,
                                                              frequency=100)
}

모델 아키텍처를 정의하고 어떤 레이어를 잘라내기 할지 지정합니다. 구조적 잘라내기는 선택한 모델의 레이어에 따라 적용됩니다.

아래 예시에서, 일부 레이어만 잘라내기 했습니다. 두 번째 `Conv2D`와 첫 번째 `Dense` 레이어를 잘라내기 했습니다.

첫 번째 `Conv2D`는 구조적으로 자를 수 없다는 점에 유의하십시오. 구조적으로 잘라내기 하려면, 입력 채널이 2개 이상이어야 합니다. 대신, 무작위 잘라내기로 첫 번째 `Conv2D` 레이어를 잘라내기 했습니다.

In [ ]:
model = keras.Sequential([
    prune_low_magnitude(
        keras.layers.Conv2D(
            32, 5, padding='same', activation='relu',
            input_shape=(28, 28, 1),
            name="pruning_sparsity_0_5"),
        **pruning_params_sparsity_0_5),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    prune_low_magnitude(
        keras.layers.Conv2D(
            64, 5, padding='same',
            name="structural_pruning"),
        **pruning_params_2_by_4),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    keras.layers.Flatten(),
    prune_low_magnitude(
        keras.layers.Dense(
            1024, activation='relu',
            name="structural_pruning_dense"),
        **pruning_params_2_by_4),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

모델을 훈련하고 평가합니다.

In [ ]:
batch_size = 128
epochs = 2

model.fit(
    train_images,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    verbose=0,
    callbacks=tfmot.sparsity.keras.UpdatePruningStep(),
    validation_split=0.1)

_, pruned_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Pruned test accuracy:', pruned_model_accuracy)

TensorFlow Lite 형식으로 변환할 때 잘라내기 래퍼를 삭제하여 이를 모델 안에 포함하지 않습니다.

In [ ]:
model = tfmot.sparsity.keras.strip_pruning(model)

## 모델을 tflite 형식으로 변환하기

In [ ]:
import tempfile

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

_, tflite_file = tempfile.mkstemp('.tflite')
print('Saved converted pruned model to:', tflite_file)
with open(tflite_file, 'wb') as f:
  f.write(tflite_model)

## 가중치 시각화 및 확인

이제 2x4 희소성으로 잘라내기 된 `Dense` 레이어에 가중치 구조를 시각화합니다. tflite 파일에서 가중치를 추출합니다.

In [ ]:
# Load tflite file with the created pruned model
interpreter = tf.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()

details = interpreter.get_tensor_details()

# Weights of the dense layer that has been pruned.
tensor_name = 'structural_pruning_dense/MatMul'
detail = [x for x in details if tensor_name in x["name"]]

# We need the first layer.
tensor_data = interpreter.tensor(detail[0]["index"])()

잘라내기 된 올바른 레이어를 선택했는지 확인하려면, 가중치 텐서의 형상을 프린트합니다.

In [ ]:
print(f"Shape of Dense layer is {tensor_data.shape}")

가중치 텐서의 구조는 `(2,4)` 패턴을 사용한 마지막 차원에서 희소합니다(4개 중 요소 2개는 0). 시각화를 보다 명확하게 하려면, 모든 0이 아닌 값을 1로 대체합니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# The value 24 is chosen for convenience.
width = height = 24

subset_values_to_display = tensor_data[0:height, 0:width]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(subset_values_to_display) > 0, val_ones, val_zeros)

보조 함수를 정의하여 분리 선을 그려 구조를 명확히 봅니다.

In [ ]:
def plot_separation_lines(height, width):

    block_size = [1, 4]

    # Add separation lines to the figure.
    num_hlines = int((height - 1) / block_size[0])
    num_vlines = int((width - 1) / block_size[1])
    line_y_pos = [y * block_size[0] for y in range(1, num_hlines + 1)]
    line_x_pos = [x * block_size[1] for x in range(1, num_vlines + 1)]

    for y_pos in line_y_pos:
        plt.plot([-0.5, width], [y_pos - 0.5 , y_pos - 0.5], color='w')

    for x_pos in line_x_pos:
        plt.plot([x_pos - 0.5, x_pos - 0.5], [-0.5, height], color='w')

이제 가중치 텐서의 하위 세트를 시각화합니다.

In [ ]:
plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structural pruning for Dense layer")
plt.show()

`Conv2D` 레이어에 대한 가중치를 시각화합니다. 구조적 희소성은 마지막 채널에 `Dense` 레이어와 유사하게 적용됩니다. 두 번째 `Conv2D` 레이어만 위에서 지적한 대로 구조적으로 잘라내기 됩니다.

In [ ]:
# Get weights of the convolutional layer that has been pruned with 2 by 4 sparsity.
tensor_name = 'structural_pruning/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[1]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

`Dense` 레이어의 가중치와 유사하게, 커널의 마지막 차원 구조는 (2, 4)입니다.

In [ ]:
weights_to_display = tf.reshape(tensor_data, [tf.reduce_prod(tensor_data.shape[:-1]), -1])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 1e-9, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structurally pruned weights for Conv2D layer")
plt.show()

이러한 무작위로 잘라내기 된 가중치가 어떤 모습인지 확인합니다. 가중치를 추출하고 가중치 텐서의 하위 집합을 표시합니다.

In [ ]:
# Get weights of the convolutional layer that has been pruned with random pruning.
tensor_name = 'pruning_sparsity_0_5/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[0]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

In [ ]:
weights_to_display = tf.reshape(tensor_data, [tensor_data.shape[0],tf.reduce_prod(tensor_data.shape[1:])])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 0, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Unstructed pruned weights for Conv2D layer")
plt.show()

TensorFlow 모델 최적화 툴킷은 특정 tflite 파일의 모델의 어떤 레이어가 구조적으로 잘라내기 된 가중치가 있는지 확인하는 데 사용될 수 있는 python 스크립트를 포함합니다([`check_sparsity_m_by_n.py`](https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/python/core/sparsity/keras/tools/check_sparsity_m_by_n.py)). 다음 명령어는 특정 모델에서 2x4 희소성을 확인하기 위해 이 툴을 어떻게 사용하는지 설명합니다.

In [ ]:
! python3 ./tensorflow_model_optimization/python/core/sparsity/keras/tools/check_sparsity_m_by_n.py --model_tflite=pruned_model.tflite --m_by_n=2,4
